In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.express as px
import datetime

# Load the data
data = pd.read_csv('electricity-consumption.csv')

# Display the first few rows of the dataset
print(data.head())

data.drop(columns=['prectotcorr', 'date_2'], inplace=True)

# Display summary statistics
print(data.describe())

# Check for missing values
print(data.isnull().sum())


     date_1  day  month  year  temp2(c)  temp2_max(c)  temp2_min(c)  \
0  01-01-18    1      1  2018     19.11         24.48         13.78   
1  02-01-18    2      1  2018     18.24         23.16         15.28   
2  03-01-18    3      1  2018     15.78         22.65         11.52   
3  04-01-18    4      1  2018     13.78         20.32          8.19   
4  05-01-18    5      1  2018     13.87         20.79          8.24   

   temp2_ave(c)  suface_pressure(pa)  wind_speed50_max(m/s)  \
0        19.130               101.08                   5.05   
1        19.220               100.94                   6.20   
2        17.085               101.12                   6.96   
3        14.255               101.34                   5.91   
4        14.515               101.17                   6.02   

   wind_speed50_min(m/s)  wind_speed50_ave(m/s)  prectotcorr  \
0                   0.23                  2.640         0.00   
1                   1.59                  3.895         0.01   
2 

In [10]:
data.dropna(inplace=True)
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1894 entries, 0 to 1945
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date_1                 1894 non-null   object 
 1   day                    1894 non-null   int64  
 2   month                  1894 non-null   int64  
 3   year                   1894 non-null   int64  
 4   temp2(c)               1894 non-null   float64
 5   temp2_max(c)           1894 non-null   float64
 6   temp2_min(c)           1894 non-null   float64
 7   temp2_ave(c)           1894 non-null   float64
 8   suface_pressure(pa)    1894 non-null   float64
 9   wind_speed50_max(m/s)  1894 non-null   float64
 10  wind_speed50_min(m/s)  1894 non-null   float64
 11  wind_speed50_ave(m/s)  1894 non-null   float64
 12  total_demand(mw)       1894 non-null   float64
 13  max_generation(mw)     1894 non-null   float64
dtypes: float64(10), int64(3), object(1)
memory usage: 222.0+ KB


In [4]:
# Create a dashboard with multiple visualizations using Plotly subplots

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create subplots
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=("Total Demand vs Temperature", 
                                    "Wind Speed vs Total Demand",
                                    "Surface Pressure vs Total Demand",
                                    "Monthly Average Demand"))

# 1. Scatter plot: Total Demand vs Temperature
fig.add_trace(
    go.Scatter(x=data['temp2_ave(c)'], y=data['total_demand(mw)'], 
               mode='markers', name='Demand vs Temp'),
    row=1, col=1
)

# 2. Scatter plot: Wind Speed vs Total Demand
fig.add_trace(
    go.Scatter(x=data['wind_speed50_ave(m/s)'], y=data['total_demand(mw)'], 
               mode='markers', name='Demand vs Wind'),
    row=1, col=2
)

# 3. Scatter plot: Surface Pressure vs Total Demand
fig.add_trace(
    go.Scatter(x=data['suface_pressure(pa)'], y=data['total_demand(mw)'], 
               mode='markers', name='Demand vs Pressure'),
    row=2, col=1
)

# 4. Bar plot: Monthly Average Demand
monthly_demand = data.groupby('month')['total_demand(mw)'].mean().reset_index()
fig.add_trace(
    go.Bar(x=monthly_demand['month'], y=monthly_demand['total_demand(mw)'], 
           name='Monthly Avg Demand'),
    row=2, col=2
)

# Update layout
fig.update_layout(height=800, width=1200, title_text="Electricity Consumption Dashboard")
fig.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_xaxes(title_text="Wind Speed (m/s)", row=1, col=2)
fig.update_xaxes(title_text="Surface Pressure (Pa)", row=2, col=1)
fig.update_xaxes(title_text="Month", row=2, col=2)
fig.update_yaxes(title_text="Total Demand (MW)", row=1, col=1)
fig.update_yaxes(title_text="Total Demand (MW)", row=1, col=2)
fig.update_yaxes(title_text="Total Demand (MW)", row=2, col=1)
fig.update_yaxes(title_text="Average Demand (MW)", row=2, col=2)

# Show the plot
fig.show()


In [9]:
# Import necessary libraries if not already imported
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                    subplot_titles=("Max Generation over Time", "Total Demand over Time"))

# Line plot for date_1 vs max_generation
fig.add_trace(
    go.Scatter(x=data['year'], y=data['max_generation(mw)'], 
               mode='lines', name='Max Generation'),
    row=1, col=1
)

# Line plot for date_1 vs total_demand
fig.add_trace(
    go.Scatter(x=data['year'], y=data['total_demand(mw)'], 
               mode='lines', name='Total Demand'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=800, width=1400, title_text="Max Generation and Total Demand over Time")
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Max Generation (MW)", row=1, col=1)
fig.update_yaxes(title_text="Total Demand (MW)", row=2, col=1)

# Show the plot
fig.show()


In [7]:
# Import necessary libraries if not already imported
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate monthly averages for demand and generation
monthly_avg = data.groupby('month').agg({
    'total_demand(mw)': 'mean',
    'max_generation(mw)': 'mean'
}).reset_index()

# Create subplots: 1 row, 2 columns
fig = make_subplots(rows=1, cols=2, 
                    specs=[[{'type':'domain'}, {'type':'xy'}]],
                    subplot_titles=('Average Monthly Demand Distribution', 
                                    'Demand vs Generation Trend'))

# Add pie chart for average monthly demand
fig.add_trace(go.Pie(
    labels=monthly_avg['month'],
    values=monthly_avg['total_demand(mw)'],
    name='Avg Monthly Demand',
    hole=0.4,
    hoverinfo='label+percent+value',
    textinfo='label+percent'
), row=1, col=1)

# Add line chart for demand vs generation trend
fig.add_trace(go.Scatter(
    x=monthly_avg['month'],
    y=monthly_avg['total_demand(mw)'],
    name='Avg Demand',
    line=dict(color='blue', width=2)
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=monthly_avg['month'],
    y=monthly_avg['max_generation(mw)'],
    name='Avg Max Generation',
    line=dict(color='red', width=2)
), row=1, col=2)

# Update layout
fig.update_layout(
    title='Electricity Demand and Generation Analysis',
    height=600,
    width=1200
)

# Update axes for the line chart
fig.update_xaxes(title_text='Month', row=1, col=2)
fig.update_yaxes(title_text='Megawatts (MW)', row=1, col=2)

# Show the plot
fig.show()

In [8]:
def create_dashboard_plots():
    # Create the first plot (Temperature vs Demand)
    fig1 = px.scatter(data, x='temp2_ave(c)', y='total_demand(mw)', 
                      title='Temperature vs Electricity Demand',
                      labels={'temp2_ave(c)': 'Average Temperature (°C)', 
                              'total_demand(mw)': 'Total Demand (MW)'})

    # Create the second plot (Monthly Average Demand and Generation)
    fig2 = make_subplots(rows=1, cols=2, 
                         specs=[[{'type':'domain'}, {'type':'xy'}]],
                         subplot_titles=('Average Monthly Demand Distribution', 
                                         'Demand vs Generation Trend'))

    # Add pie chart for average monthly demand
    fig2.add_trace(go.Pie(
        labels=monthly_avg['month'],
        values=monthly_avg['total_demand(mw)'],
        name='Avg Monthly Demand',
        hole=0.4,
        hoverinfo='label+percent+value',
        textinfo='label+percent'
    ), row=1, col=1)

    # Add line chart for demand vs generation trend
    fig2.add_trace(go.Scatter(
        x=monthly_avg['month'],
        y=monthly_avg['total_demand(mw)'],
        name='Avg Demand',
        line=dict(color='blue', width=2)
    ), row=1, col=2)

    fig2.add_trace(go.Scatter(
        x=monthly_avg['month'],
        y=monthly_avg['max_generation(mw)'],
        name='Avg Max Generation',
        line=dict(color='red', width=2)
    ), row=1, col=2)

    # Update layout for fig2
    fig2.update_layout(
        title='Electricity Demand and Generation Analysis',
        height=600,
        width=1200
    )

    # Update axes for the line chart in fig2
    fig2.update_xaxes(title_text='Month', row=1, col=2)
    fig2.update_yaxes(title_text='Megawatts (MW)', row=1, col=2)

    # Create the third plot (Wind Speed vs Demand)
    fig3 = px.scatter(data, x='wind_speed50_ave(m/s)', y='total_demand(mw)', 
                      title='Wind Speed vs Electricity Demand',
                      labels={'wind_speed50_ave(m/s)': 'Average Wind Speed (m/s)', 
                              'total_demand(mw)': 'Total Demand (MW)'})

    # Create the fourth plot (Time Series of Demand)
    fig4 = px.line(data, x='date_1', y='total_demand(mw)', 
                   title='Electricity Demand Over Time',
                   labels={'date_1': 'Date', 'total_demand(mw)': 'Total Demand (MW)'})
    return fig1, fig2, fig3, fig4  # Return all the figures you want to display